In [35]:
import numpy as np
from sklearn import preprocessing
import python_speech_features as mfcc

import pandas as pd

import numpy as np
from scipy.io.wavfile import read
from sklearn.mixture import GMM 

import warnings
warnings.filterwarnings("ignore")

import six
from six.moves import cPickle as pickle



In [14]:



def calculate_delta(array):
    """Calculate and returns the delta of given feature vector matrix"""

    rows,cols = array.shape
    deltas = np.zeros((rows,20))
    N = 2
    for i in range(rows):
        index = []
        j = 1
        while j <= N:
            if i-j < 0:
                first = 0
            else:
                first = i-j
            if i+j > rows -1:
                second = rows -1
            else:
                second = i+j
            index.append((second,first))
            j+=1
        deltas[i] = ( array[index[0][0]]-array[index[0][1]] + (2 * (array[index[1][0]]-array[index[1][1]])) ) / 10
    return deltas

def extract_features(audio,rate):
    """extract 20 dim mfcc features from an audio, performs CMS and combines 
    delta to make it 40 dim feature vector"""    
    
    mfcc_feat = mfcc.mfcc(audio,rate, 0.025, 0.01,20,appendEnergy = True)
    
    mfcc_feat = preprocessing.scale(mfcc_feat)
    delta = calculate_delta(mfcc_feat)
    combined = np.hstack((mfcc_feat,delta)) 
    return combined


In [29]:

##/home/bigdata/scripts/DCMWork/development_set/

#path to training data
source   = "/home/bigdata/scripts/development_set/"   

#path where training speakers will be saved
dest = "speaker_models/"

In [32]:
source.strip()
dest.strip()

'speaker_models/'

In [45]:
##/home/bigdata/scripts/DCMWork/development_set/

#path to training data
source   = "/home/bigdata/scripts/development_set/"   

#path where training speakers will be saved
dest = "speaker_models/"

train_file = "development_set_enroll.txt"        


file_paths = open(train_file,'r')


file_pd = pd.read_csv(train_file,sep="%",header=None)
file_pd=file_pd.rename( columns={0: "fpath"})

file_pd








,fpath
0,anthonyschaller-20071221-\wav\a0491.wav
1,anthonyschaller-20071221-\wav\a0492.wav
2,anthonyschaller-20071221-\wav\a0493.wav
3,anthonyschaller-20071221-\wav\a0494.wav
4,anthonyschaller-20071221-\wav\a0495.wav
5,Apple_Eater-20091026-yul\wav\a0052.wav
6,Apple_Eater-20091026-yul\wav\a0053.wav
7,Apple_Eater-20091026-yul\wav\a0054.wav
8,Apple_Eater-20091026-yul\wav\a0055.wav
9,Apple_Eater-20091026-yul\wav\a0056.wav


In [33]:



count = 1

# Extracting features for each speaker (5 files per speakers)
features = np.asarray(())
for path in file_paths:    
    stipedpath = source.strip() + path.strip()   
    print (stipedpath)
    
    # read the audio
    sr,audio = read(stipedpath)
    
    # extract 40 dimensional MFCC & delta MFCC features
    vector   = extract_features(audio,sr)
    
    if features.size == 0:
        features = vector
    else:
        features = np.vstack((features, vector))
    # when features of 5 files of speaker are concatenated, then do model training
    if count == 5:    
        gmm = GMM(n_components = 16, n_iter = 200, covariance_type='diag',n_init = 3)
        gmm.fit(features)
        
        # dumping the trained gaussian model
        picklefile = path.split("-")[0]+".gmm"
        cPickle.dump(gmm,open(dest + picklefile,'w'))
        print ('+ modeling completed for speaker:',picklefile," with data point = ",features.shape)    
        features = np.asarray(())
        count = 0
    count = count + 1

/home/bigdata/scripts/development_set/anthonyschaller-20071221-\wav\a0491.wav


FileNotFoundError: [Errno 2] No such file or directory: '/home/bigdata/scripts/development_set/anthonyschaller-20071221-\\wav\\a0491.wav'